# Configuration prioritization using QAOA

We want to extend our findings from the qaoa_mincost_sat notebook by adding functionality for creating an ordered list of configurations sorted by ascending configuration cost.
We will be using the same mincost SAT instance as before as an example.


The cost for each valid configuration is as follows:

| Feature Vector | Config. Cost | Feature Vector | Config. Cost |
|----------------|--------------|----------------|--------------|
| 01 01 01       | 80           | 01 10 00       | 45           |
| 10 01 01       | 90           | 10 10 00       | 55           |
| 11 01 01       | 110          | 11 10 00       | 75           |
| 01 10 01       | 55           | 01 01 11       | 90           |
| 10 10 01       | 65           | 10 01 11       | 100          |
| 11 10 01       | 95           | 11 01 11       | 120          |
| 01 01 00       | 70           | 01 10 11       | 65           |
| 10 01 00       | 80           | 10 10 11       | 75           |
| 11 01 00       | 100          | 11 10 11       | 95           |

In [ ]:
# Notebook Setup
from IPython.core import page
page.page = print

# Imports used for examples
import qaoa_quantum_deflation as quantum_deflation

In [ ]:
# cost and sat individually
from qubovert import boolean_var

# define binary vars
x1, x2, x3, x4, x5, x6 = boolean_var('x1'), boolean_var('x2'), boolean_var('x3'), boolean_var('x4'), boolean_var('x5'), boolean_var('x6')

# SAT Penalty
alpha_sat = 1000 # 1e6

# SAT QUBO
sat_model = alpha_sat * (2 - x1 - x2 - x3 - x4 + x5 + x1 * x2 + 2 * x3 * x4 - x5 * x6)

# Cost QUBO
cost_model = 30*x1 + 20*x2 + 25*x3 + 50*x4 + 30*x5 + 10*x6

# Combine models
combined_model = sat_model +  cost_model
combined_hamiltonian = combined_model.to_quso()

In [ ]:
# Config cost for each valid config
config_cost = {"101010": 80, "101001":  90, "101011": 110,
               "100110": 55, "100101":  65, "100111":  95,
               "001010": 70, "001001":  80, "001011": 100,
               "000110": 45, "000101":  55, "000111":  75,
               "111010": 90, "111001": 100, "111011": 120,
               "110110": 65, "110101":  75, "110111":  95}

# Adjusting the Hamiltonian

In order to create an ordered list of configurations with ascending cost, we run the QAOA algorithm multiple times and adjust the hamiltonian for each run to exclude the config that was previously measured with the highest probability.
In more detail, that means that, in order to determine the $k$-th best configuration, we need to exclude every configuration before the $k$-th best in our hamiltonian.
We do this by using different deflation methods which are explained below.

## Variational Quantum Deflation
For Variational Quantum Deflation (VQD) as described by [Higgott et al.](https://arxiv.org/abs/1805.08138), we can determine the effective hamiltonian $H_k$ where the first to $k$-th best configurations are deflated using the following formular:

$H_k := H + \sum_{i = 0}^{k - 1} \beta_i | i \rangle \langle i |$

where $i$ is the previously found $i$-th configuration of our optimization problem.

To guarantee a minimum hamiltonian energy at $E_k$ for the $k$-th configuration, we need to choose $\beta_i > E_k - E_i$.
We achieve this by running the algorithm multiple times and adjusting (i.e. doubling) $\beta_i$ starting from a predetermined value until we find a $\beta_i$ which is big enough.

## Projection Deflation
For Projection Deflation (PD) as described by [Higgott et al.](https://arxiv.org/abs/1805.08138), we can determine the effective hamiltonian $H_k$ using the following formular:

$H_k = A^\top_k (H - E') A_k$

where:

$A_k := \prod_{i = 0}^{k - 1} (1 - | i \rangle \langle i |) \approx 1 - \sum_{i = 0}^{k - 1} | i \rangle \langle i |$

and $i$ is the same as for VQD above and $E' > E_k$.


In [ ]:
from qaoa_mincost_sat import problem_circuit
import configproblem.qaoa.qaoa_mixer as mixer
import configproblem.qaoa.qaoa_parameter_optimization as parameter_optimization
# deflation factor start value for vqd: 2600
# deflation factor start value for pd: 800
prioritized_list = quantum_deflation.config_prioritization(problem_circuit, mixer.standard_mixer, parameter_optimization.get_optimizer('COBYLA'), combined_hamiltonian, 6, 3, 2600, strategy='vqd', debug_output=True)

for item in prioritized_list:
    # Check config validity
    if config_cost.keys().__contains__(item):
        print("Valid config " + str(item) + " with cost: " + str(config_cost[item]))
    else:
        print("Invalid config")

## Conclusion

We found that the deflation methods above are not applicable to our problem.
We want to eliminate specific configurations from our hamiltonian, while the different deflation methods eliminate eigenstates from the hamiltonian.
This means that in order to apply such methods to our problem, we would need a hamiltonian where each possible config is an eigenvector thus requiring a hamiltonian of dimension $2^n$ and therefore also $2^n$ qubits where $n$ is the length of our feature vector.
This approach wouldn't scale well and is therefore not feasible.